In [ ]:
import numpy as np
import pandas as pd
import os
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import random
import math

In [ ]:
currDir = os.getcwd()
gameDF = pd.read_csv(currDir+'\\data\\gameData.csv')
gameDF = gameDF.sort_values(['date', 'gameId']).reset_index(drop=True)

gameDF['homeWins'], gameDF['homeLosses'], gameDF['visitorWins'], gameDF['visitorLosses'] = np.zeros(len(gameDF)), np.zeros(len(gameDF)), np.zeros(len(gameDF)), np.zeros(len(gameDF))

gameDF

In [ ]:
def recalcWinsLosses(df):
    returnDF = pd.DataFrame(columns = df.columns)
    
    wins, losses = {}, {}
    for idNum in np.unique(df['homeTeamID']):
        wins.update({idNum : 0})
        losses.update({idNum : 0})
        
    lastSeason = 5
    for i, row in df.iterrows():
        if i % 100 == 0:
            print(i)
            clear_output(wait=True)
            
        thisSeason = int(row['gameId']/100000)%10
        if thisSeason != lastSeason:
            wins, losses = {}, {}
            for idNum in np.unique(df['homeTeamID']):
                wins.update({idNum : 0})
                losses.update({idNum : 0})
        lastSeason = thisSeason

        homeID = row['homeTeamID']
        visitorID = row['visitorTeamID']
        
        row['homeWins'], row['homeLosses'] = wins[homeID], losses[homeID]
        row['visitorWins'], row['visitorLosses'] = wins[visitorID], losses[visitorID]
        if row['hWin'] == 1:
            wins.update({homeID : wins[homeID] + 1})
            losses.update({visitorID : losses[visitorID] + 1})
        else:
            wins.update({visitorID : wins[visitorID] + 1})
            losses.update({homeID : losses[homeID] + 1})
        returnDF = returnDF.append(row)
    
    return returnDF
        
gameDF = recalcWinsLosses(gameDF)

gameDF[['date', 'gameId', 'homeTeamID', 'homeWins', 'homeLosses', 'homeWStreak', 'visitorTeamID', 'visitorWins', 'visitorLosses', 'visitorWStreak', 'hWin']] = gameDF[['date', 'gameId', 'homeTeamID', 'homeWins', 'homeLosses', 'homeWStreak', 'visitorTeamID', 'visitorWins', 'visitorLosses', 'visitorWStreak', 'hWin']].astype('int32')

gameDF

In [ ]:
sortedHome = gameDF.sort_values(['homeTeamID', 'date']).reset_index(drop=True, inplace=False)

columns = ['date', 'gameId', 'teamID', 'wins', 'losses', 'streak']
for col in sortedHome.columns[6:25]:
    columns.append(col[5:])
columns.append('hWin')
    
homeCols = ['homeTeamID', 'homeWins', 'homeLosses', 'homeWStreak', 'home_fastBreakPoints', 'home_pointsInPaint', 'home_biggestLead',
       'home_pointsOffTurnovers', 'home_points', 'home_fgm', 'home_fga', 'home_ftm', 'home_fta', 'home_tpm', 'home_tpa', 'home_offReb',
       'home_defReb', 'home_assists', 'home_pFouls', 'home_steals', 'home_turnovers', 'home_blocks', 'home_plusMinus']
awayCols = ['visitorTeamID', 'visitorWins', 'visitorLosses', 'visitorWStreak', 'visitor_fastBreakPoints', 'visitor_pointsInPaint',
       'visitor_biggestLead', 'visitor_pointsOffTurnovers', 'visitor_points', 'visitor_fgm', 'visitor_fga', 'visitor_ftm', 'visitor_fta', 'visitor_tpm',
       'visitor_tpa', 'visitor_offReb', 'visitor_defReb', 'visitor_assists', 'visitor_pFouls', 'visitor_steals', 'visitor_turnovers', 'visitor_blocks', 'visitor_plusMinus']

data = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]

print(len(homeCols), len(awayCols))
for i, row in gameDF.iterrows():
    data[0].append(row['date'])
    data[1].append(row['gameId'])
    data[25].append(row['hWin'])
    data[0].append(row['date'])
    data[1].append(row['gameId'])
    data[25].append(row['hWin'])
    for x in range(23):
        data[x+2].append(row[homeCols[x]])
        data[x+2].append(row[awayCols[x]])
        
dataDict = {}
for x in range(len(data)):
    dataDict.update({columns[x] : data[x]})


masterDF = pd.DataFrame(dataDict)

masterDF[['date', 'gameId', 'teamID', 'wins', 'losses', 'streak', 'hWin']] = masterDF[['date', 'gameId', 'teamID', 'wins', 'losses', 'streak', 'hWin']].astype('int32')

masterDF

In [ ]:
sortedMaster = masterDF.sort_values(by=['teamID', 'date']).reset_index()
avgCols = list(sortedMaster.columns[7:-1])
isGood = []

dict0, dict1, dict2, dict3 = {'empty': False}, {'empty':True}, {'empty': True}, {'empty' : True}

currTeamID, currSeason = 0, 0

data = {}
for col in avgCols:
    data.update({col : []})

for i, row in sortedMaster.iterrows():
    if i % 10 == 0:
        print(i)
        clear_output(wait=True)
    if currSeason == int(row['gameId']/100000)%10 and currTeamID == row['teamID']:
            
            if dict3['empty']:
                isGood.append(False)
                
                dict3 = dict2
                dict2 = dict1
                dict1 = dict0
                dict0 = {'empty': False}

                for col in avgCols:
                    dict0.update({col : row[col]})
                    data[col].append(-1)
            else:
                isGood.append(True)
            
                dict3 = dict2
                dict2 = dict1
                dict1 = dict0
                dict0 = {'empty': False}
                
                for col in avgCols:
                    dict0.update({col : row[col]})
                    data[col].append(float((3*dict1[col])+(2*dict2[col])+dict3[col])/6)   
    else:
        currSeason = int(row['gameId']/100000)%10
        currTeamID = row['teamID']
        isGood.append(False)
        dict0, dict1, dict2, dict3 = {'empty' : False}, {'empty': False}, {'empty': True}, {'empty' : True}
        for col in avgCols:
            dict0.update({col : row[col]})
            data[col].append(-1)
            

for col in avgCols:
    sortedMaster[col] = data[col]
sortedMaster = sortedMaster[isGood].sort_values('index').reset_index(drop=True)
sortedMaster.drop(['index'], axis=1, inplace=True)

sortedMaster

In [ ]:
counts = sortedMaster['gameId'].value_counts()
isGood = []

for i, row in sortedMaster.iterrows():
    gameID = row['gameId']
    
    if counts[gameID] == 2:
        isGood.append(True)
    else:
        isGood.append(False)

sortedMaster = sortedMaster[isGood].reset_index(drop=True, inplace=False)

sortedMaster

In [ ]:
columns = list(gameDF.columns)
columns[2], columns[3], columns[4], columns[5] = 'home_teamID', 'home_wins', 'home_losses', 'home_streak'
columns[25], columns[26], columns[27], columns[28] = 'visitor_teamID', 'visitor_wins', 'visitor_losses', 'visitor_streak'
data = {}

for col in columns:
    data.update({col : []})

    
for i, row in sortedMaster.iterrows():
    
    if i % 2 == 0:
        data['date'].append(row['date'])
        data['gameId'].append(row['gameId'])
        data['hWin'].append(row['hWin']) 
        for col in columns[2:25]:
            data[col].append(row[col[5:]])
    
    else:
        for col in columns[25:-1]:
            data[col].append(row[col[8:]])   


threeAvgWeightedDF = pd.DataFrame(data=data)

threeAvgWeightedDF[['date', 'gameId', 'home_teamID', 'home_wins', 'home_losses', 'home_streak', 'visitor_teamID', 'visitor_wins', 'visitor_losses', 'visitor_streak', 'hWin']] = threeAvgWeightedDF[['date', 'gameId', 'home_teamID', 'home_wins', 'home_losses', 'home_streak', 'visitor_teamID', 'visitor_wins', 'visitor_losses', 'visitor_streak', 'hWin']].astype('int32')


threeAvgWeightedDF

In [ ]:
badIndices = []
for i, row in threeAvgWeightedDF.iterrows():
    if i % 1000 == 0:
        print(i)
    for col in row.keys():
        if np.isnan(row[col]) and i not in badIndices:
            badIndices.append(i)
            
threeAvgWeightedDF = threeAvgWeightedDF.drop(badIndices)
threeAvgWeightedDF

In [ ]:
threeAvgWeightedDF.to_csv(currDir+'\\data\\threeAvgsWeighted.csv', index=False)